In [106]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import spotipy.util as util
import json
import pprint

client_id = 'd6f8bc651cae4a63b32ffd9d201ceff7'
client_secret = '5bd943039c2046b39fce9310232cae96'
username = 'literallyanything'
scope = 'user-library-read'
redirect_uri = 'http://localhost:7778/callback'
token_obj = util.prompt_for_user_token(
    username=username, 
    scope=scope, 
    client_id=client_id,   
    client_secret=client_secret,
    redirect_uri=redirect_uri
)

In [107]:
def get_user_library(token):
    user_saved_tracks = []
    get_next_track_set = True
    offset = 0
    headers = {
        'Authorization': 'Bearer {}'.format(token)
    }
    while get_next_track_set == True:
        params = {'limit' : 50, 'offset' : offset}
        response = requests.get('https://api.spotify.com/v1/me/tracks', headers=headers, params=params)
        tracks_in_set = json.loads(response.text)
        tracks_in_set = tracks_in_set['items']
        if len(tracks_in_set) > 0:
            user_saved_tracks = user_saved_tracks + tracks_in_set
            offset = offset + 50
        elif len(tracks_in_set) == 0:
            get_next_track_set = False
            
    saved_tracks_formatted = []
    for obj in user_saved_tracks:
        added_at = obj['added_at']
        track = obj['track']
        
        track_formatted = {
            'track_id' : track['id'],
            'saved_at' : added_at,
            'track_name' : track['name'],
            'primary_artist_id' : track['artists'][0]['id'],
            'all_artists' : [artist['id'] for artist in track['artists']],
            'album_id' : track['album']['id'],
            'duration_ms' : track['duration_ms'],
            'is_explicit' : track['explicit'],
            'popularity': track['popularity'],
            'track_number' : track['track_number']
        }
        saved_tracks_formatted.append(track_formatted)
        
    saved_tracks_df = pd.DataFrame(saved_tracks_formatted)
    list_of_track_ids = list(saved_tracks_df['track_id'])
    
    all_track_features = []
    lower_bound = 0
    upper_bound = 99
    for interval in range(0,len(list_of_track_ids),100):
        track_features = requests.get('https://api.spotify.com/v1/audio-features/', headers=headers, params={'ids': ','.join(list_of_track_ids[lower_bound:upper_bound])})
        track_features = json.loads(track_features.text)['audio_features']
        all_track_features = all_track_features + track_features
        lower_bound = upper_bound
        upper_bound += 100
    
    track_features_df = pd.DataFrame(all_track_features)
    
    saved_tracks_df = saved_tracks_df.merge(track_features_df, how='left', left_on='track_id', right_on='id')
    
    saved_tracks_df = saved_tracks_df[
        [
            'album_id', 'all_artists', 'duration_ms_x', 'is_explicit', 'popularity',
            'primary_artist_id', 'track_id', 'track_name', 'track_number',
            'acousticness', 'danceability','energy', 'instrumentalness', 
            'key', 'liveness', 'loudness','mode', 'speechiness', 'tempo', 
            'time_signature', 'valence','saved_at'
        ]
    ]
    
    saved_tracks_df = saved_tracks_df.rename(columns={'duration_ms_x' : 'duration_ms'})
    
    saved_tracks_df = saved_tracks_df[['track_id','saved_at','track_name','primary_artist_id','all_artists','album_id','track_number','duration_ms','popularity','is_explicit','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence']]
    
    return saved_tracks_df
                
        

In [108]:
library = get_user_library(token_obj)
library

,track_id,saved_at,track_name,primary_artist_id,all_artists,album_id,track_number,duration_ms,popularity,is_explicit,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1kcfGBb6kSrGqNIMW7rAlB,2020-07-18T20:50:52Z,Oh Devil,7sgWBYtJpblXpJl2lU5WVs,[7sgWBYtJpblXpJl2lU5WVs],4ncZdrGiO459vrUU1yYCCA,5,217586,58,False,...,0.532,0.034500,7,0.1760,-7.188,0,0.0593,96.991,4,0.7010
1,2ZjdNFPXtVdIMJeDBJBHj8,2020-07-18T20:30:35Z,Fake You,7FU0xCgmSYQEiBeevUqQ4S,[7FU0xCgmSYQEiBeevUqQ4S],44EVGCWT8hQyf0QPLy3DOf,1,223170,47,False,...,0.606,0.133000,6,0.1070,-8.885,1,0.0582,163.942,4,0.2600
2,0iOZD91DEQm3nuTEk5Bt8k,2020-07-18T14:42:22Z,Alone,5Mq7iqCWBzofK39FBqblNc,[5Mq7iqCWBzofK39FBqblNc],6mPNjjOl2GHbP4E90l7bJe,5,216866,45,False,...,0.615,0.000001,8,0.1590,-7.339,1,0.0614,174.182,4,0.1920
3,5teYCP3pROESCEaASCVf5w,2020-07-18T03:26:56Z,You're Too Precious,53KwLdlmrlCelAZMaLVZqU,[53KwLdlmrlCelAZMaLVZqU],0hs2pdNq5QXYWsEcAFVO9q,1,223900,65,False,...,0.477,0.366000,1,0.1030,-9.458,1,0.1400,149.890,3,0.2970
4,1kX5gKYWhFTiXwQxyQDYoo,2020-07-16T18:47:19Z,So Long,3jLDJWlhuDcbq1DTlWp0NM,[3jLDJWlhuDcbq1DTlWp0NM],71pWNnr2LM2SS4wTJOavFq,5,220841,32,False,...,0.352,0.021300,0,0.0872,-10.873,0,0.0273,96.116,3,0.2670
5,63V3K1kB4gTFeK1Oj02xsh,2020-07-16T18:47:13Z,All Aboard,3jLDJWlhuDcbq1DTlWp0NM,[3jLDJWlhuDcbq1DTlWp0NM],2tm020f8C1bf1FvZ5CC3GU,5,168680,11,False,...,0.139,0.033800,2,0.1230,-18.264,1,0.0462,113.033,4,0.3350
6,2RB2FLPBELP6ku0uDZld4d,2020-07-16T17:31:48Z,Holiday,3jLDJWlhuDcbq1DTlWp0NM,[3jLDJWlhuDcbq1DTlWp0NM],7MabqJDlen13npNSTKpYde,1,205333,28,False,...,0.162,0.060600,8,0.1040,-14.411,1,0.0867,108.061,3,0.1740
7,1HBXXiiegRl0TpzgikBBTP,2020-07-16T17:24:54Z,Shirim,1S0vL284jxZYKtZQ2jsQ2X,[1S0vL284jxZYKtZQ2jsQ2X],0jolpH4eoj3kSEVg9mvZXr,7,286680,53,False,...,0.710,0.445000,0,0.2540,-10.992,1,0.0607,101.163,4,0.5280
8,68Z57OWBBp6p4ieKYgz8hJ,2020-07-16T17:03:23Z,SERPENTWITHFEET_ALTADENA _88 BPM,1U0sIzpRtDkvu1hXXzxh60,"[1U0sIzpRtDkvu1hXXzxh60, 1O9iHQjrVuiAYOJFCBeFSl]",00bR0PfkGVAizXpN05DYrs,8,202979,36,False,...,0.504,0.000002,1,0.1990,-7.805,0,0.0389,87.999,4,0.2810
9,7eO7Aj3iOQFIEVhxDYaCnM,2020-07-16T16:59:19Z,Giver,1gSte6U8m2rJabGxR9DDow,"[1gSte6U8m2rJabGxR9DDow, 439Dw5xIEnUHlgIu1QS2uF]",47ekjrJD7sm62ynppcJoEW,1,207961,35,False,...,0.545,0.010200,2,0.0680,-4.984,1,0.2190,76.960,4,0.2400
